In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
import re
import seaborn as sns
sns.set_theme(style="whitegrid")

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import save_model

In [2]:
physical_devices = tf.config.list_physical_devices("GPU")
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
data_path = pathlib.Path("../data/raw/PlantVillage")

In [4]:
features = np.load("features.npy")
labels = np.load("labels.npy")

In [5]:
labels

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [9]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.2)

In [10]:
train_features.shape

(43444, 128, 128, 3)

In [11]:
train_labels.shape

(43444, 2)

In [12]:
test_features.shape

(10861, 128, 128, 3)

In [13]:
test_labels.shape

(10861, 2)

In [ ]:
input_shape=(128,128,3)
classes = 2
epoch = 3
batch_size = 64

inception_v3 = InceptionV3(weights=None, input_shape=input_shape, classes=classes)

# inception_v3.summary()

inception_v3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

early_stop= EarlyStopping(monitor='val_loss',patience=10)

mcp_save = ModelCheckpoint('mdl_inception_v3_1.hdf5', save_best_only=True, monitor='val_loss', mode='min')

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=10, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

inception_v3.fit(train_features, train_labels,
          epochs=epoch,
          batch_size=batch_size,
          validation_data=(test_features,test_labels), 
          callbacks=[early_stop, learning_rate_reduction])


In [19]:
filepath = '../models/inception_v3'
save_model(inception_v3, filepath)

metrics=pd.DataFrame(inception_v3.history.history)
metrics

INFO:tensorflow:Assets written to: ../models/inception_v3\assets


,loss,accuracy,val_loss,val_accuracy,lr
0,0.237571,0.905971,0.286113,0.912531,0.001
1,0.092147,0.966301,0.131209,0.951017,0.001
2,0.054803,0.980734,0.121908,0.954056,0.001


In [20]:
results = inception_v3.evaluate(test_features,test_labels)
print(f"test acc = {results[1]}\ntest loss = {results[0]}")

10861/10861 [==============================] - 33s 3ms/sample - loss: 0.1219 - accuracy: 0.9541
test acc = 0.9540557861328125
test loss = 0.1219079796345396
